In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.dirname(sys.path[0]))

In [2]:
import pandas as pd
from databases.postgresql import PostgresDB, Session

In [3]:
dates = [
'2023/12/28',
'2023/12/29',
'2023/12/30',
'2023/12/31',
'2024/01/01',
'2024/01/02',
'2024/01/03',
'2024/01/04',
'2024/01/05',
'2024/01/06',
'2024/01/07',
'2024/01/08',
'2024/01/09'
]

dates_pairs = list()
for i in range(0, len(dates) - 1):
    dates_pairs.append([dates[i], dates[i+1]])
dates_pairs

[['2023/12/28', '2023/12/29'],
 ['2023/12/29', '2023/12/30'],
 ['2023/12/30', '2023/12/31'],
 ['2023/12/31', '2024/01/01'],
 ['2024/01/01', '2024/01/02'],
 ['2024/01/02', '2024/01/03'],
 ['2024/01/03', '2024/01/04'],
 ['2024/01/04', '2024/01/05'],
 ['2024/01/05', '2024/01/06'],
 ['2024/01/06', '2024/01/07'],
 ['2024/01/07', '2024/01/08'],
 ['2024/01/08', '2024/01/09']]

In [4]:
schemas = ['chain_0x38', 'chain_0x1', 'chain_0xfa', 'chain_0x89',
            'chain_0xa', 'chain_0xa4b1', 'chain_0xa86a', 'chain_0x2b6653dc']

tables_sizes = dict()

tables = ['amount_in_out',	'dapp_interaction',	'smart_contract',	'token_decimals',	'token_transfer', 'wrapped_token']

for schema in schemas:
    tables_sizes[schema] = pd.read_csv(f'../data/{schema}_tables_size.csv', index_col='time')
    tables_sizes[schema] = tables_sizes[schema].loc[dates, tables]
    tables_sizes[schema] = tables_sizes[schema].apply(pd.to_numeric)



In [5]:
def subtract_rows(df: pd.DataFrame, date_1: str, date_2):
    _diffs = (df.loc[date_2] - df.loc[date_1])
    _diffs = _diffs.to_frame().T
    _diffs['time'] = date_2
    _diffs.set_index('time', inplace=True)
    return _diffs

In [6]:
differences = dict()
for schema in schemas:
    # differences[schema] = pd.DataFrame()
    for _i, _dates_pair in enumerate(dates_pairs):
        _diffs = subtract_rows(df=tables_sizes[schema], date_2=_dates_pair[1], date_1=_dates_pair[0])
        if _i < 1:
            differences[schema] = _diffs
        else:
            differences[schema] = pd.concat([differences[schema], _diffs])
            
    differences[schema]['sum'] = differences[schema].sum(axis=1)

In [7]:
_id = 0
print(schemas[_id])
differences[schemas[_id]]

chain_0x38


,amount_in_out,dapp_interaction,smart_contract,token_decimals,token_transfer,wrapped_token,sum
time,,,,,,,
2023/12/29,40280064,171638784,0,204800,285523968,39149568,536797184
2023/12/30,89194496,139264,0,172032,1143644160,36511744,1269661696
2023/12/31,-40263680,428204032,0,163840,58204160,26714112,473022464
2024/01/01,-83853312,192020480,0,155648,2575785984,56164352,2740273152
2024/01/02,-12976128,199270400,0,155648,6324224,39067648,231841792
2024/01/03,188416,225107968,0,188416,571310080,60907520,857702400
2024/01/04,360448,134627328,0,155648,100638720,48103424,283885568
2024/01/05,5472256,266117120,0,172032,1097916416,26738688,1396416512
2024/01/06,-11591680,184737792,0,180224,34660352,53846016,261832704


In [8]:
for schema in schemas:
    # print(f"{schema}: {differences[schema].loc['2024/01/03'] / (2**30)}")
    print(f"{schema}: {differences[schema]['sum'][dates[-1]] / (2**30)}")

chain_0x38: 0.1716766357421875
chain_0x1: 0.01978302001953125
chain_0xfa: 0.00501251220703125
chain_0x89: 0.161102294921875
chain_0xa: 0.0044708251953125
chain_0xa4b1: 0.35504913330078125
chain_0xa86a: 0.05002593994140625
chain_0x2b6653dc: 0.22891998291015625


In [9]:
total_increases = dict()
for _date in dates[1:]:
    _sum_all = 0
    for schema in schemas:
        _sum_all += differences[schema].loc[_date, 'sum']
    total_increases[_date] = _sum_all / (2**30)
    
total_increases

{'2023/12/29': 2.6023483276367188,
 '2023/12/30': 3.5559539794921875,
 '2023/12/31': 3.98284912109375,
 '2024/01/01': 5.17938232421875,
 '2024/01/02': 4.002708435058594,
 '2024/01/03': 2.9643096923828125,
 '2024/01/04': 4.2373199462890625,
 '2024/01/05': 3.7888641357421875,
 '2024/01/06': 2.9591445922851562,
 '2024/01/07': 1.476226806640625,
 '2024/01/08': 1.1546783447265625,
 '2024/01/09': 0.9960403442382812}